# 《分布式计算》2021-2022学年第2学期期末考试

提示与建议：

1. 优先完成更有把握的题目；
2. 对于简答题，回答核心要点即可，不要提供冗长的陈述；
3. 对于编程题，优先完成整体的框架，有时间富余时再进行细节上的调试；
4. 对于综合题，每一小问的题目本身可能就是对后续问题的提示；
5. 可参考课件及以往的作业，但注意与考题的差异，不要生搬硬套。

### 第1题：简答题（20分）

**(a) 结合本课程的内容，简述梯度下降法、牛顿法、L-BFGS 和 ADMM 算法各自的适用范围和优缺点。（10分）**

**(b) 在使用 Spark RDD 进行分布式计算时，是否划分的分区数越多，总的计算时间越短？有哪些可能影响计算效率的因素？（10分）**

### 第2题：编程题（30分）

蒙特卡洛方法的一大应用是近似计算积分。例如，如果要计算

$$
I=\int_{-\infty}^{+\infty} f(x)g(x)\mathrm{d}x,
$$

其中 $g(x)$ 是一个随机变量的密度函数，那么根据期望公式，我们有

$$
I=\mathbb{E}_{X}[f(X)],\quad X\sim g(x).
$$

根据大数定律，如果我们产生服从 $g(x)$ 分布的独立同分布随机数 $X_1,\ldots,X_n$，那么

$$
I\approx \frac{1}{n}\sum_{i=1}^n f(X_i).
$$

**(a) 根据以上信息，利用蒙特卡洛方法和 Numpy 编写程序近似计算积分**

$$I_1=\int_{-\infty}^{+\infty} \left\{[\sin(x)]^2+\cos(x)\right\}\cdot \frac{1}{\sqrt{2\pi}}e^{-x^2/2}\mathrm{d}x,$$

**生成随机数时以自己的学号作为随机数种子，取样本量 $n=10000$。（10分）**

> 提示1：标准正态分布 $N(0,1)$ 的密度函数为
>
> $$
\phi(x)=\frac{1}{\sqrt{2\pi}}e^{-\frac{x^2}{2}}.
$$

> 提示2：计算结果可以利用数值积分进行验证，例如
>
> ```py
> # 计算 h(x)=exp(-|x|) 在 (-Inf, +Inf) 上的积分
> import numpy as np
> import scipy.integrate as integrate
> 
> integrate.quad(lambda x: np.exp(-np.abs(x)), -np.inf, np.inf)
> # 结果的第一个元素是积分值，第二个是误差估计
> ```
> ```
> (2.0000000000000004, 1.1685213993527393e-10)
> ```
>
> 该程序可用来帮助验证计算结果，但无需包含在最终的回答中。

**(b) 利用 PySpark 实现一个分布式计算 (a) 中积分的程序，原理如下：**

我们采用分布式的方法并行模拟大量的随机数。考虑将所有的随机数分成10组，其中1-5组每组生成 10000 个随机数，6-10组每组生成 5000 个点，每组独立产生随机数并计算各自的 $f(X_i)$ 求和，最后将所有10组的结果汇总并得出最终对积分的估计。为了使结果可重复，第 $i$ 组在生成随机数时使用 $i$ 作为随机数种子。PySpark 使用本地模式，开启 4 个 CPU 核心。**（10分）**

**(c) 利用 (b) 中的方法，近似计算积分**

$$I_2=\int_{-\infty}^{+\infty} \left\{[\sin(x)]^2+\cos(x)\right\}\cdot e^{-(x-1)^2}\mathrm{d}x.$$
**（10分）**

> 提示3：正态分布 $N(\mu,\sigma^2)$ 的密度函数为
> $$
h(x)=\frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{(x-\mu)^2}{2\sigma^2}}.
$$

### 第3题：综合题（50分）

Least Absolute Deviation (LAD) 是一种稳健回归方法，它与线性回归中的最小二乘方法（OLS）类似，但用残差的绝对值之和来替代平方和。为了与 ADMM 算法的记号匹配，我们用 $A\in\mathbb{R}^{n\times p}$ 表示自变量矩阵，$b\in\mathbb{R}^n$ 表示因变量向量，要估计的回归系数为 $x\in\mathbb{R}^p$。于是 LAD 的目标函数为

$$\Vert Ax-b\Vert_1,$$

其中 $\Vert v\Vert_1$ 表示向量 $v=(v_1,\ldots,v_n)'$ 的 $L^1$ 范数，即 $\Vert v\Vert_1=|v_1|+\cdots+|v_n|$。

LAD 可以改写为 ADMM 的形式：$f(x)=0$，$g(z)=||z||_1$，$B=-I$，$c=b$。其迭代公式为

$$
\begin{align*}
x^{k+1} & =(A'A)^{-1}A'(b+z^{k}-u^{k})\\
z^{k+1} & =S_{1/\rho}(Ax^{k+1}-b+u^{k})\\
u^{k+1} & =u^{k}+Ax^{k+1}-z^{k+1}-b,
\end{align*}
$$

其中 $S_{\kappa}(a)$ 为 soft-thresholding 运算符，定义为

$$
S_{\kappa}(a)=\begin{cases}
a-\kappa, & a>\kappa\\
0, & |a|\le\kappa\\
a+\kappa, & a<-\kappa
\end{cases},
$$

一种紧凑的表达是 $S_{\kappa}(a)=\mathrm{sign}(a)\cdot\max\{0,|a|-\kappa\}$。

相应地，原问题残差为 $r^{k+1}=Ax^{k+1}-z^{k+1}-b$，对偶问题残差为 $s^{k+1}=-\rho A'(z^{k+1}-z^{k})$。

**(a) $x$、$z$ 和 $u$ 各自的维度是多少？（3分）**

**(b) 编写函数 `update_z()` 来实现 $z$ 的更新，函数需返回更新后的 $z$ 变量。（7分）**

**(c) 编写函数 `update_x()` 来实现 $x$ 的更新，函数需返回更新后的 $x$ 变量。（10分）**

**(d) 在 LAD 的 ADMM 算法中，$x$ 需要反复进行迭代，但注意到 $A$ 是固定的。如果假定 $A'A$ 是正定矩阵，那么应该采取怎样的方法，使得反复进行的 $x$ 更新更加高效？简述即可，无需在此处提供代码实现。（5分）**

**(e) 利用 ADMM 算法求解 LAD 问题**

$$\min_x\ \Vert Ax-b\Vert_1,$$

**并将其封装成一个函数：**

```python
admm_lad(A, b, rho=1.0, maxit=10000, eps=1e-3, verbose=0)
```

**其中 `A` 是 $n\times p$ 的自变量矩阵，`b` 是 $n\times 1$ 的因变量向量，`rho` 是 ADMM 算法的 $\rho$ 参数，`maxit` 是最大迭代次数，`eps` 是 ADMM 收敛的残差临界值，`verbose` 表示是否输出迭代信息，如果 $>0$，则每隔 1000 次迭代打印出当前的两类残差，如果 $\le 0$ 则不输出任何信息。**

**函数需返回两个量，第一个表示实际使用的迭代次数，第二个表示估计的回归系数。该函数可利用前面小题中已经编写的函数，也可重新编写，且应实现（d）中给出的方法。（15分）**

利用模拟训练集数据测试上述编写的函数：

In [ ]:
np.random.seed(123)
n = 1000
p = 30
Atrain = np.random.normal(size=(n, p))
x = np.random.normal(size=p)
btrain = Atrain.dot(x) + np.random.normal(size=n)
x

In [ ]:
admm_lad(Atrain, btrain, rho=1.0, maxit=10000, eps=1e-3, verbose=1)

In [ ]:
admm_lad(Atrain, btrain, rho=0.1, maxit=10000, eps=1e-3, verbose=0)

**(f) 利用 PySpark 来并行地考察 $\rho$ 参数对算法收敛速度的影响。取 $\rho=0.1,0.2,\ldots,1.0$，找出使得迭代次数最少的 $\rho$。（10分）**